# Using hyperopt-sklearn with `Bench`


[hyperopt-sklearn](https://github.com/hyperopt/hyperopt-sklearn) is a popular library for tuning many "classical" ML models. The convenience this library brings is that it has predefined hyperparameter grids that are automatically tuned when one calls `fit`.

One difficulty is that it currently does not have a convenient interface for providing custom validation sets (see [this issue](https://github.com/hyperopt/hyperopt-sklearn/issues/152)). We will use this as one other example for how to implement a model for benchmarking with `mofdscribe`.


In [1]:
from mofdscribe.bench import LogkHCO2IDBench
from mofdscribe.datasets.core_dataset import CoREDataset
from hpsklearn import (
    HyperoptEstimator,
    gaussian_process_regressor,
    lightgbm_regression,
    power_transformer,
    standard_scaler,
    xgboost_regression,
)

from mofdscribe.splitters import HashSplitter
import numpy as np
from copy import deepcopy


In [2]:
ds = CoREDataset()

FEATURES = list(ds.available_features)

TARGET = "outputs.logKH_CO2"


2022-08-07 16:25:09.726 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:25:09.762 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104


In [7]:
class TunedXBoost:
    def __init__(self, features):
        self.model = HyperoptEstimator(regressor=xgboost_regression("mymodel"))
        self.features = features

    def tune(self, idx, y):
        tune_splitter = HashSplitter(self.ds.get_subset(idx))
        # we will now use a simple split in two parts,
        # however, you could also use a k-fold in the tune method
        models = []
        for train_idx_, valid_idx_ in tune_splitter.k_fold(5):
            train_idx = idx[train_idx_]
            valid_idx = idx[valid_idx_]

            train_x, train_y = self.ds._df.iloc[train_idx][self.features], y[train_idx_]
            valid_x, valid_y = self.ds._df.iloc[valid_idx][self.features], y[valid_idx_]

            # we concatenate train and validation data
            # but make sure to turn of shuffling and use the last fraction of the data for validation
            x = np.concatenate([train_x, valid_x])
            y = np.concatenate([train_y, valid_y])

            valid_frac = len(valid_x) / len(x)

            model = deepcopy(self.model)
            model.fit(x, y, cv_shuffle=False, n_folds=None, valid_size=valid_frac)

            models.append((model._best_loss, model, model._best_learner))

        models = sorted(models, key=lambda x: x[0]) 
        self.model = models[0][1]

    def fit(self, idx, structures, y):
        self.tune(idx, y)
        X = self.ds._df.iloc[idx][self.features]
        self.model.fit(X, y)

    def predict(self, idx, structures):
        X = self.ds._df.iloc[idx][self.features]
        pred = self.model.predict(X)
        return pred


In [8]:
model = TunedXBoost(FEATURES)


In [12]:
bench = LogkHCO2IDBench(model, name="xgboost-hyperopt", debug=False, patch_in_ds=True)


2022-08-07 16:54:49.212 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:54:49.364 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:54:53.772 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:54:53.806 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:54:53.819 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 1.0, q (0, 0.25, 0.5, 0.75, 1)


In [13]:
report = bench.bench()


2022-08-07 16:54:54.247 | DEBUG    | mofdscribe.bench.mofbench:_score:230 - K-fold round 0, 1690 train points, 414 test points
2022-08-07 16:54:58.571 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:54:58.606 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:54:58.637 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 1.0, q (0, 0.25, 0.5, 0.75, 1)


100%|██████████| 10/10 [01:07<00:00, 67.74s/trial, best loss: 1.026138470266063]
[(0.694418136430368, hyperopt_estimator(ex_preprocs=(),
                   preprocessing=<hyperopt.pyll.base.Apply object at 0x3015f9190>,
                   regressor=<hyperopt.pyll.base.Apply object at 0x3015f9250>,
                   space=<hyperopt.pyll.base.Apply object at 0x2c9430520>), XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.7720770462728171, colsample_bynode=1,
             colsample_bytree=0.85288795632054, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             gamma=0.8170613897195499, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.03777359868922444, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=8, max_leaves=0,
             min_child_weight=49, missing=nan, monotone_constraints='()',
      

In [11]:
report

BenchResult(start_time=datetime.datetime(2022, 8, 7, 14, 26, 50, 724208, tzinfo=datetime.timezone.utc), end_time=datetime.datetime(2022, 8, 7, 14, 41, 22, 5793, tzinfo=datetime.timezone.utc), metrics=RegressionMetricCollection(regression_metrics=[RegressionMetrics(mean_squared_error=2.2541407682886128, mean_absolute_error=1.2180623255137877, r2_score=0.033736962545019455, max_error=2.6369412290583982, mean_absolute_percentage_error=1.1498512872694546, top_5_in_top_5=1, top_10_in_top_10=1, top_50_in_top_50=1, top_100_in_top_100=1, top_500_in_top_500=1), RegressionMetrics(mean_squared_error=1.1809820371171182, mean_absolute_error=0.9447372616054199, r2_score=-0.8369989405136833, max_error=1.855924517571582, mean_absolute_percentage_error=0.36642558493148253, top_5_in_top_5=1, top_10_in_top_10=1, top_50_in_top_50=1, top_100_in_top_100=1, top_500_in_top_500=1), RegressionMetrics(mean_squared_error=1.3269701986068339, mean_absolute_error=0.9784902795550539, r2_score=-3.740793935068651, max_